In [181]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from datetime import datetime
import gc
import copy
import os
from openpyxl import load_workbook

# 载入重要因子列表

In [286]:
# files vs names：资金利率类 vs 狭义流动性类；资金策略类 vs 资产策略类；
names = ['经济前景类指标', '投资类指标', '工业类指标', '进出口类指标', '房地产类指标', 
         '消费类指标', '价格指数类指标', '统计局食品类指标', '狭义流动性指标', '广义流动性指标', 
         '内外利差类指标', '汇率类指标', '跨境资本流动类指标', '财政政策类指标', '货币政策类指标', 
         '其他政府监管类指标', '资产策略类指标']
cols = {}
for i in [1, 2, 3, 5, 7, 10]:
    cols[i] = []
    data = pd.read_excel(str(i) + '年期结果.xlsx', sheet_name = '重要因子')
    data[['子项目', '子项目.1']] = data[[ '子项目', '子项目.1']].fillna(method = 'ffill')
    for name in names:
        cols[i] = cols[i] + list(data[(data['子项目'] == name) & (data['对应变量'] != '数据量过少，剔除')]['指标'].values) + list(data[(data['子项目.1'] == name) & (data['对应变量'] != '数据量过少，剔除')]['指标'].values)
    cols[i] = list(set(cols[i]))

# 导出重要因子数据

In [108]:
files = ['经济前景类指标.xlsx', '投资类指标.xlsx', '工业类指标.xlsx', '进出口类指标.xlsx', '房地产类指标.xlsx', 
         '消费类指标.xlsx', '价格指数类指标.xlsx', '统计局食品类指标.xlsx', '资金利率类指标.xlsx', '广义流动性类指标.xlsx', 
         '内外利差类指标_1.xlsx', '内外利差类指标_2.xlsx', '内外利差类指标_3.xlsx', '内外利差类指标_5.xlsx', '内外利差类指标_7.xlsx', 
         '内外利差类指标_10.xlsx', '汇率类指标.xlsx', '跨境资本流动类指标.xlsx', '财政政策类指标.xlsx', '货币政策类指标.xlsx', 
         '其他政府监管1类指标.xlsx', '其他政府监管2类指标.xlsx', '资金策略1类指标.xlsx', '资金策略2类指标.xlsx']

In [164]:
def loading_file(i, sheet = 0):
    print('Loading file {} in 24 ...'.format(i+1))
    data = pd.read_excel(files[i], skipfooter = 1, sheet_name = sheet)
    data.set_index('描述', inplace = True)
    y = treasury[['描述', name]]
    y.set_index('描述', inplace = True)
    if i == 20:
        data = data.join(y[name], how = 'outer')
        data[file_name + '-SHIBOR:3个月'] = data[name] - data.pop('SHIBOR:3个月')
        data[file_name + '-FR007'] = data[name] - data.pop('回购定盘利率:7天(FR007)')
        data[file_name + '-GC007'] = data.pop(name) - data.pop('GC007:收盘价')
    if i == 22:
        data = data.join(y[name], how = 'outer')
        data[file_name + '-同业存单到期收益率(AAA+):1年'] = data[name] - data.pop('同业存单到期收益率(AAA+):1年')
        data[file_name + '-短期融资券到期收益率(AAA+):1年'] = data[name] - data.pop('短期融资券到期收益率(AAA+):1年')
        data[file_name + '-贷款基础利率(LPR):1年'] = data[name] - data.pop('贷款基础利率(LPR):1年')
        if num == '1':
            data[file_name + '-中债国开债到期收益率:1年'] = data[name] - data.pop('中债国开债到期收益率:1年')
            data[file_name + '-中债商业银行普通债到期收益率(AAA):1年'] = data[name] - data.pop('中债商业银行普通债到期收益率(AAA):1年')
            data[file_name + '-中债企业债到期收益率(AAA):1年'] = data[name] - data.pop('中债企业债到期收益率(AAA):1年')
            data[file_name + '-中债铁道债到期收益率:1年'] = data[name] - data.pop('中债铁道债到期收益率:1年')
            data[file_name + '-中债地方政府债到期收益率(AAA):1年'] = data.pop(name) - data.pop('中债地方政府债到期收益率(AAA):1年')
            data = data[[file_name + '-中债国开债到期收益率:1年', file_name + '-中债商业银行普通债到期收益率(AAA):1年', 
                         file_name + '-中债企业债到期收益率(AAA):1年', file_name + '-中债铁道债到期收益率:1年',
                         file_name + '-中债地方政府债到期收益率(AAA):1年', file_name + '-同业存单到期收益率(AAA+):1年', 
                         file_name + '-短期融资券到期收益率(AAA+):1年', file_name + '-贷款基础利率(LPR):1年']]
        if num == '2':
            data[file_name + '-中债国开债到期收益率:2年'] = data[name] - data.pop('中债国开债到期收益率:2年')
            data[file_name + '-中债商业银行普通债到期收益率(AAA):2年'] = data[name] - data.pop('中债商业银行普通债到期收益率(AAA):2年')
            data[file_name + '-中债企业债到期收益率(AAA):2年'] = data[name] - data.pop('中债企业债到期收益率(AAA):2年')
            data[file_name + '-中债铁道债到期收益率:2年'] = data[name] - data.pop('中债铁道债到期收益率:2年')
            data[file_name + '-中债地方政府债到期收益率(AAA):2年'] = data.pop(name) - data.pop('中债地方政府债到期收益率(AAA):2年')
            data = data[[file_name + '-中债国开债到期收益率:2年', file_name + '-中债商业银行普通债到期收益率(AAA):2年', 
                         file_name + '-中债企业债到期收益率(AAA):2年', file_name + '-中债铁道债到期收益率:2年',
                         file_name + '-中债地方政府债到期收益率(AAA):2年', file_name + '-同业存单到期收益率(AAA+):1年', 
                         file_name + '-短期融资券到期收益率(AAA+):1年', file_name + '-贷款基础利率(LPR):1年']]
        if num == '3':
            data[file_name + '-中债国开债到期收益率:3年'] = data[name] - data.pop('中债国开债到期收益率:3年')
            data[file_name + '-中债商业银行普通债到期收益率(AAA):3年'] = data[name] - data.pop('中债商业银行普通债到期收益率(AAA):3年')
            data[file_name + '-中债企业债到期收益率(AAA):3年'] = data[name] - data.pop('中债企业债到期收益率(AAA):3年')
            data[file_name + '-中债铁道债到期收益率:3年'] = data[name] - data.pop('中债铁道债到期收益率:3年')
            data[file_name + '-中债地方政府债到期收益率(AAA):3年'] = data.pop(name) - data.pop('中债地方政府债到期收益率(AAA):3年')
            data = data[[file_name + '-中债国开债到期收益率:3年', file_name + '-中债商业银行普通债到期收益率(AAA):3年', 
                         file_name + '-中债企业债到期收益率(AAA):3年', file_name + '-中债铁道债到期收益率:3年',
                         file_name + '-中债地方政府债到期收益率(AAA):3年', file_name + '-同业存单到期收益率(AAA+):1年', 
                         file_name + '-短期融资券到期收益率(AAA+):1年', file_name + '-贷款基础利率(LPR):1年']]
        if num == '5':
            data[file_name + '-中债国开债到期收益率:5年'] = data[name] - data.pop('中债国开债到期收益率:5年')
            data[file_name + '-中债商业银行普通债到期收益率(AAA):5年'] = data[name] - data.pop('中债商业银行普通债到期收益率(AAA):5年')
            data[file_name + '-中债企业债到期收益率(AAA):5年'] = data[name] - data.pop('中债企业债到期收益率(AAA):5年')
            data[file_name + '-中债铁道债到期收益率:5年'] = data[name] - data.pop('中债铁道债到期收益率:5年')
            data[file_name + '-中债地方政府债到期收益率(AAA):5年'] = data.pop(name) - data.pop('中债地方政府债到期收益率(AAA):5年')
            data = data[[file_name + '-中债国开债到期收益率:5年', file_name + '-中债商业银行普通债到期收益率(AAA):5年', 
                         file_name + '-中债企业债到期收益率(AAA):5年', file_name + '-中债铁道债到期收益率:5年',
                         file_name + '-中债地方政府债到期收益率(AAA):5年', file_name + '-同业存单到期收益率(AAA+):1年', 
                         file_name + '-短期融资券到期收益率(AAA+):1年', file_name + '-贷款基础利率(LPR):1年']]
        if num == '7':
            data[file_name + '-中债国开债到期收益率:7年'] = data[name] - data.pop('中债国开债到期收益率:7年')
            data[file_name + '-中债商业银行普通债到期收益率(AAA):7年'] = data[name] - data.pop('中债商业银行普通债到期收益率(AAA):7年')
            data[file_name + '-中债企业债到期收益率(AAA):7年'] = data[name] - data.pop('中债企业债到期收益率(AAA):7年')
            data[file_name + '-中债铁道债到期收益率:7年'] = data[name] - data.pop('中债铁道债到期收益率:7年')
            data[file_name + '-中债地方政府债到期收益率(AAA):7年'] = data.pop(name) - data.pop('中债地方政府债到期收益率(AAA):7年')
            data = data[[file_name + '-中债国开债到期收益率:7年', file_name + '-中债商业银行普通债到期收益率(AAA):7年', 
                         file_name + '-中债企业债到期收益率(AAA):7年', file_name + '-中债铁道债到期收益率:7年',
                         file_name + '-中债地方政府债到期收益率(AAA):7年', file_name + '-同业存单到期收益率(AAA+):1年', 
                         file_name + '-短期融资券到期收益率(AAA+):1年', file_name + '-贷款基础利率(LPR):1年']]
        if num == '10':
            data[file_name + '-中债国开债到期收益率:10年'] = data[name] - data.pop('中债国开债到期收益率:10年')
            data[file_name + '-中债商业银行普通债到期收益率(AAA):10年'] = data[name] - data.pop('中债商业银行普通债到期收益率(AAA):10年')
            data[file_name + '-中债企业债到期收益率(AAA):10年'] = data[name] - data.pop('中债企业债到期收益率(AAA):10年')
            data[file_name + '-中债铁道债到期收益率:10年'] = data[name] - data.pop('中债铁道债到期收益率:10年')
            data[file_name + '-中债地方政府债到期收益率(AAA):10年'] = data.pop(name) - data.pop('中债地方政府债到期收益率(AAA):10年')
            data = data[[file_name + '-中债国开债到期收益率:10年', file_name + '-中债商业银行普通债到期收益率(AAA):10年', 
                         file_name + '-中债企业债到期收益率(AAA):10年', file_name + '-中债铁道债到期收益率:10年',
                         file_name + '-中债地方政府债到期收益率(AAA):10年', file_name + '-同业存单到期收益率(AAA+):1年', 
                         file_name + '-短期融资券到期收益率(AAA+):1年', file_name + '-贷款基础利率(LPR):1年']]
    data.reset_index(inplace = True)
    return data

In [184]:
def excelAddSheet(dataframe, outfile, name, ind = True):
    writer = pd.ExcelWriter(outfile, enginge='openpyxl')
    if os.path.exists(outfile) != True:
        dataframe.to_excel(writer, name, index = ind, encoding = 'utf_8_sig')
    else:
        book = load_workbook(writer.path)
        writer.book = book
        dataframe.to_excel(excel_writer = writer, sheet_name = name, index = ind, encoding = 'utf_8_sig')
    writer.save()
    writer.close()

In [310]:
treasury = pd.read_excel('国债收益率.xlsx', skipfooter = 1)
for i in [1, 2, 3, 5, 7, 10]:
    num = str(i)
    name = '中债国债到期收益率:' + str(i) + '年'
    file_name = str(i) + '年期国债收益率'
    my_data = treasury[['描述', name]]
    for j in range(24):
        new = loading_file(j)
        col_use = ['描述'] + list(new.columns.difference(my_data.columns))
        my_data = my_data.merge(new[col_use], on = '描述', how = 'outer', copy = False, sort = '描述')
    my_data = my_data[['描述', name] + cols[i]]
    my_data = my_data[(my_data['描述'] < '2018-08-01') & (my_data['描述'] > '2002-01-03')]
    my_data[name].fillna(method = 'ffill', inplace = True)
    my_data['year'] = my_data['描述'].apply(lambda x: x.year)
    my_data['month'] = my_data['描述'].apply(lambda x: x.month)
    # 月度化
    for col in my_data.columns[2:-2]:
        my_data[col].fillna(method = 'ffill', inplace = True)
    my_data.drop_duplicates(['year', 'month'], keep = 'last', inplace = True)
    my_data.set_index(['year', 'month'], drop = True, inplace = True)
    my_data.dropna(inplace = True)
    # 自变量：变化率；因变量：差分
    out = pd.DataFrame(my_data[name])
    diff = pd.Series(my_data[name].diff(1).shift(-1).dropna(), name = name + '_diff')
    out = out.join(diff, how = 'inner')
    for col in list(my_data.columns[2:]):
        mid = (my_data[col].diff(1) / my_data[col].shift(1))
        mid[np.isinf(mid)] = np.nan
        mid.fillna(method = 'ffill', inplace = True)
        out = out.join(mid, how = 'inner')
        out.dropna(inplace = True)
    excelAddSheet(out, 'for_model_added_dates_v2.xlsx', file_name, ind = True)

Loading file 1 in 24 ...
Loading file 2 in 24 ...
Loading file 3 in 24 ...
Loading file 4 in 24 ...
Loading file 5 in 24 ...
Loading file 6 in 24 ...
Loading file 7 in 24 ...
Loading file 8 in 24 ...
Loading file 9 in 24 ...
Loading file 10 in 24 ...
Loading file 11 in 24 ...
Loading file 12 in 24 ...
Loading file 13 in 24 ...
Loading file 14 in 24 ...
Loading file 15 in 24 ...
Loading file 16 in 24 ...
Loading file 17 in 24 ...
Loading file 18 in 24 ...
Loading file 19 in 24 ...
Loading file 20 in 24 ...
Loading file 21 in 24 ...
Loading file 22 in 24 ...
Loading file 23 in 24 ...
Loading file 24 in 24 ...
Loading file 1 in 24 ...
Loading file 2 in 24 ...
Loading file 3 in 24 ...
Loading file 4 in 24 ...
Loading file 5 in 24 ...
Loading file 6 in 24 ...
Loading file 7 in 24 ...
Loading file 8 in 24 ...
Loading file 9 in 24 ...
Loading file 10 in 24 ...
Loading file 11 in 24 ...
Loading file 12 in 24 ...
Loading file 13 in 24 ...
Loading file 14 in 24 ...
Loading file 15 in 24 ...
Load

In [186]:
treasury = pd.read_excel('国债收益率.xlsx', skipfooter = 1)
for i in [1, 2, 3, 5, 7, 10]:
    num = str(i)
    name = '中债国债到期收益率:' + str(i) + '年'
    file_name = str(i) + '年期国债收益率'
    my_data = treasury[['描述', name]]
    for j in range(24):
        new = loading_file(j)
        col_use = ['描述'] + list(new.columns.difference(my_data.columns))
        my_data = my_data.merge(new[col_use], on = '描述', how = 'outer', copy = False, sort = '描述')
    my_data = my_data[['描述', name] + cols[i]]
    my_data.dropna(subset = [name], inplace = True)
    # 数据对齐+月度化
    my_data['year'] = my_data['描述'].apply(lambda x: x.year)
    my_data['month'] = my_data['描述'].apply(lambda x: x.month)
    my_data.fillna(method = 'ffill', inplace = True)
    my_data.drop_duplicates(['year', 'month'], keep = 'last', inplace = True)
    my_data.dropna(inplace = True)
    my_data.reset_index(drop = True, inplace = True)
    # 自变量：变化率；因变量：差分
    out = pd.DataFrame((my_data[name].diff(1).dropna()[1:]).reset_index(drop = True))
    out.join
    for col in list(my_data.columns[2:-2]):
        mid = (my_data[col].diff(1) / my_data[col]).fillna(0)
        mid.fillna(0, inplace = True)
        mid[np.isinf(mid)] = 0
        out[col] = mid[1:].reset_index(drop = True)
    excelAddSheet(out, 'for_model.xlsx', file_name, ind = False)

Loading file 1 in 24 ...
Loading file 2 in 24 ...
Loading file 3 in 24 ...
Loading file 4 in 24 ...
Loading file 5 in 24 ...
Loading file 6 in 24 ...
Loading file 7 in 24 ...
Loading file 8 in 24 ...
Loading file 9 in 24 ...
Loading file 10 in 24 ...
Loading file 11 in 24 ...
Loading file 12 in 24 ...
Loading file 13 in 24 ...
Loading file 14 in 24 ...
Loading file 15 in 24 ...
Loading file 16 in 24 ...
Loading file 17 in 24 ...
Loading file 18 in 24 ...
Loading file 19 in 24 ...
Loading file 20 in 24 ...
Loading file 21 in 24 ...
Loading file 22 in 24 ...
Loading file 23 in 24 ...
Loading file 24 in 24 ...
Loading file 1 in 24 ...
Loading file 2 in 24 ...
Loading file 3 in 24 ...
Loading file 4 in 24 ...
Loading file 5 in 24 ...
Loading file 6 in 24 ...
Loading file 7 in 24 ...
Loading file 8 in 24 ...
Loading file 9 in 24 ...
Loading file 10 in 24 ...
Loading file 11 in 24 ...
Loading file 12 in 24 ...
Loading file 13 in 24 ...
Loading file 14 in 24 ...
Loading file 15 in 24 ...
Load